<a href="https://colab.research.google.com/github/1324fgg/GraphAlign/blob/main/dyf_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First section: connect to drive and prepare the environment


In [ ]:
#Connect file to /content/drive, actually you colab folder path is /content/drive/MyDrive/colab
from google.colab import drive
drive.mount('/content/drive')

#Change the current path to our work directory
import os
path = "/content/drive/MyDrive/ColabNotebooks/socialScienceProject"
os.chdir(path)
os.listdir

!nvidia-smi
!pip install ogb
!pip install torch
!pip install dgl==1.1.0 -f https://data.dgl.ai/wheels/repo.html

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/bin/bash: line 1: nvidia-smi: command not found
Looking in links: https://data.dgl.ai/wheels/repo.html


In [ ]:
from ogb.nodeproppred import DglNodePropPredDataset

mag_dataset = DglNodePropPredDataset(name = "ogbn-mag")
mag_split_idx = mag_dataset.get_idx_split()
mag_train_idx, mag_valid_idx, mag_test_idx = mag_split_idx["train"], mag_split_idx["valid"], mag_split_idx["test"]
mag_graph, mag_label = mag_dataset[0] # graph: dgl graph object, label: torch tensor of shape (num_nodes, num_tasks)

In [ ]:
print(mag_graph)

Graph(num_nodes={'author': 1134649, 'field_of_study': 59965, 'institution': 8740, 'paper': 736389},
      num_edges={('author', 'affiliated_with', 'institution'): 1043998, ('author', 'writes', 'paper'): 7145660, ('paper', 'cites', 'paper'): 5416271, ('paper', 'has_topic', 'field_of_study'): 7505078},
      metagraph=[('author', 'institution', 'affiliated_with'), ('author', 'paper', 'writes'), ('paper', 'paper', 'cites'), ('paper', 'field_of_study', 'has_topic')])


In [ ]:
print(mag_graph)
print(mag_label)

Graph(num_nodes={'author': 1134649, 'field_of_study': 59965, 'institution': 8740, 'paper': 736389},
      num_edges={('author', 'affiliated_with', 'institution'): 1043998, ('author', 'writes', 'paper'): 7145660, ('paper', 'cites', 'paper'): 5416271, ('paper', 'has_topic', 'field_of_study'): 7505078},
      metagraph=[('author', 'institution', 'affiliated_with'), ('author', 'paper', 'writes'), ('paper', 'paper', 'cites'), ('paper', 'field_of_study', 'has_topic')])
{'paper': tensor([[246],
        [131],
        [189],
        ...,
        [266],
        [289],
        [  1]])}


In [ ]:
from ogb.nodeproppred import DglNodePropPredDataset

arxiv_dataset = DglNodePropPredDataset(name = "ogbn-arxiv")
arxiv_split_idx = arxiv_dataset.get_idx_split()
arxiv_train_idx, arxiv_valid_idx, arxiv_test_idx = arxiv_split_idx["train"], arxiv_split_idx["valid"], arxiv_split_idx["test"]
arxiv_graph, arxiv_label = arxiv_dataset[0] # graph: dgl graph object, label: torch tensor of shape (num_nodes, num_tasks)

In [ ]:
print(arxiv_graph)

Graph(num_nodes=169343, num_edges=1166243,
      ndata_schemes={'year': Scheme(shape=(1,), dtype=torch.int64), 'feat': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={})


# Second Section: we need to output the basic information of this 2 dataset, and sample a samller dataset from the mag with 2w nodes.

## 2.1 Here we generate basic information of 2 dataset

In [ ]:

import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import random

# ------------------------- Utility Functions -------------------------
def plot_degree_distribution(degrees, title, filename, loglog=False):

    degree_counts = Counter(degrees)
    x, y = zip(*degree_counts.items())

    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, s=20, alpha=0.6)
    plt.title(title, fontsize=14)
    plt.xlabel('Degree' if not loglog else 'Degree (log)')
    plt.ylabel('Frequency' if not loglog else 'Frequency (log)')

    if loglog:
        plt.xscale('log')
        plt.yscale('log')
        plt.grid(True, which="both", ls="--")
    else:
        plt.grid(True)

    plt.savefig(f"{filename}.png", dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()


def analyze_network(G, network_name):
    # ----------------- Basic Properties -----------------
    metrics = {
        'num_nodes': G.number_of_nodes(),
        'num_edges': G.number_of_edges(),
        'avg_in_degree': np.mean([d for _, d in G.in_degree()]),
        'avg_out_degree': np.mean([d for _, d in G.out_degree()]),
        'density': nx.density(G)
    }

    # ----------------- Degree Distributions -----------------
    print(G.number_of_nodes())
    print(G.number_of_edges())

    all_degrees = list(dict(G.degree()).values())
    print(all_degrees)

    plot_degree_distribution(all_degrees,
                           f"{network_name} Degree Distribution (Linear)",
                           f"{network_name}_degree_linear")
    plot_degree_distribution(all_degrees,
                           f"{network_name} Degree Distribution (Log-Log)",
                           f"{network_name}_degree_log",
                           loglog=True)


    # ----------------- Giant Component Analysis -----------------
    if nx.is_directed(G):
        scc = max(nx.strongly_connected_components(G), key=len)
        #gain the giant component, with all the attributes of the original graph
        G_gc = G.subgraph(scc)
    else:
        gcc = max(nx.connected_components(G), key=len)
        G_gc = G.subgraph(gcc)

    metrics['gc_nodes'] = G_gc.number_of_nodes()
    print(G_gc.number_of_nodes())

    # ----------------- Path Length Distribution -----------------
    path_lengths = []

    sample_size = 50
    sampled_nodes = random.sample(list(G_gc.nodes()), sample_size)

    for node in sampled_nodes:
        lengths = nx.single_source_shortest_path_length(G_gc, node)
        print
        path_lengths.extend(lengths.values())

    plt.figure(figsize=(10,6))
    plt.hist(path_lengths, bins=30)
    plt.title(f"{network_name} Path Length Distribution")
    plt.savefig(f"{network_name}_path_lengths.png", dpi=300)
    plt.show()
    plt.close()

    metrics['avg_path_length'] = np.mean(path_lengths)

    # ----------------- Clustering Analysis -----------------
    sampled_clustering = {}

    # Convert MultiDiGraph to DiGraph
    G_simple = nx.DiGraph(G_gc)
    for node in sampled_nodes:
        sampled_clustering[node] = nx.clustering(G_simple, node)
    clustering = sampled_clustering

    plt.figure(figsize=(10,6))
    plt.hist(clustering.values(), bins=30)
    plt.title(f"{network_name} Clustering Coefficients")
    plt.savefig(f"{network_name}_clustering.png", dpi=300)
    plt.show()
    plt.close()

    metrics['avg_clustering'] = np.mean(list(clustering.values()))

    return metrics


In [ ]:
import dgl
arxiv_G = dgl.to_networkx(arxiv_graph, edge_attrs=None, node_attrs=None)
arxiv_metrics = analyze_network(arxiv_G, "arxiv")
print(arxiv_metrics)

KeyboardInterrupt: 

In [ ]:
import dgl
paper_edges = ('paper', 'cites', 'paper')
paper_graph = mag_graph.edge_type_subgraph([paper_edges])
mag_G = dgl.to_homogeneous(paper_graph)
mag_G = dgl.to_networkx(mag_G, edge_attrs=None, node_attrs=None)
mag_metrics = analyze_network(mag_G, "mag")
print(mag_metrics)

## 2.2 Here we sample a subgraph of mag containing 2w nodes.

In [ ]:
import dgl
import torch

def bfs_sampling_with_high_degree(dgl_graph, target_node_limit=200000):
    # 选择度数最高的节点作为种子节点
    all_degrees = dgl_graph.in_degrees()  # 获取节点的入度
    start_node = torch.argmax(all_degrees).item()  # 找到入度最高的节点

    # 广度优先搜索（BFS）扩展邻居节点
    visited_nodes = set()
    queue = [start_node]

    while queue and len(visited_nodes) < target_node_limit:
        node = queue.pop(0)
        if node not in visited_nodes:
            visited_nodes.add(node)
            queue.extend(dgl_graph.successors(node).tolist())  # 添加邻居节点

    # 将访问的节点创建为子图
    sampled_nodes = list(visited_nodes)
    subgraph = dgl.node_subgraph(dgl_graph, sampled_nodes)
    return subgraph

#仅保留节点和边，采用dgl对象的edge_type_subgraph的方法
paper_edges = ('paper', 'cites', 'paper')
paper_graph = mag_graph.edge_type_subgraph([paper_edges])

#从异构图转化为同质图，即节点和边都转化成统一类型，需要手动指定year和feat
mag_G = dgl.to_homogeneous(paper_graph, ndata=['year', 'feat'], edata=['reltype'])
# 示例：使用 BFS 基于高入度节点采样 2 万节点子图
sampled_graph = bfs_sampling_with_high_degree(mag_G, target_node_limit=200000)

print(mag_graph.ndata)
print(paper_graph.ndata)
print(sampled_graph.ndata)

#add mag_lables to sampled_graph
sampled_ids = sampled_graph.ndata['_ID']  # Original IDs of nodes in sampled_graph
#paper_labels = mag_label['paper'] #注意这里一定要添加paper才能通过_ID 访问标签
# Retrieve labels using original IDs
sampled_graph.ndata['y'] = mag_label['paper'][sampled_ids]  # Add labels to sampled graph

print(sampled_graph.ndata['y'].shape)

#save sampled graph
import pickle

def save_graph(graph, filename):
    with open(filename, 'wb') as f:
        pickle.dump(graph, f)
    print(f"Graph saved to {filename}")

# Save your sampled graph
save_graph(sampled_graph, "sampled_graph_mag_200000.pkl")

In [ ]:
import pickle
#load the saved sampled graph mag:
def load_graph(filename):
    with open(filename, 'rb') as f:
        graph = pickle.load(f)
    print(f"Graph loaded from {filename}")
    return graph

# Load the graph
sampled_graph = load_graph("sampled_graph_mag_200000.pkl")

# Verify the loaded graph structure
print("Node data:", sampled_graph.ndata.keys())
print("Edge data:", sampled_graph.edata.keys())


## 2.3 Then we describe the sampled subgraph of mag.

In [ ]:
#转换成network之后可以计算graph的特征并绘图
sampled_graph_nx = dgl.to_networkx(sampled_graph, edge_attrs=None, node_attrs=None)
mag_metrics = analyze_network(sampled_graph_nx, "mag_sampledgraph")

# 检查子图信息
print("Number of nodes in sampled graph:", sampled_graph.num_nodes())
print("Number of edges in sampled graph:", sampled_graph.num_edges())


In [ ]:
print(mag_graph)
print(paper_graph)
print(sampled_graph)

Here we comes to train the sampled mag graph.

# Third Section:we can train the graphsage on sampled mag graph using the original embedding.

## 3.1 First we define the architecture of GraphSAGE

In [ ]:
import torch
import torch.nn.functional as F
import dgl
from dgl.nn import SAGEConv
from sklearn.metrics import accuracy_score

# Define GraphSAGE model
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats, num_layers, dropout):
        super(GraphSAGE, self).__init__()
        self.layers = torch.nn.ModuleList()
        self.layers.append(SAGEConv(in_feats, hidden_feats, aggregator_type='mean'))
        for _ in range(num_layers - 2):
            self.layers.append(SAGEConv(hidden_feats, hidden_feats, aggregator_type='mean'))
        self.layers.append(SAGEConv(hidden_feats, out_feats, aggregator_type='mean'))
        self.dropout = dropout

    def forward(self, g, x):
        for i, layer in enumerate(self.layers):
            x = layer(g, x)
            if i != len(self.layers) - 1:  # No activation on the last layer
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

# Training loop
def train(model, graph, features, labels, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(graph, features)
    loss = F.cross_entropy(out[train_idx], labels[train_idx], label_smoothing=0.1)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluation loop
@torch.no_grad()
def evaluate(model, graph, features, labels, split_idx):
    model.eval()
    out = model(graph, features)
    preds = out.argmax(dim=1)
    train_acc = accuracy_score(labels[split_idx['train']].cpu(), preds[split_idx['train']].cpu())
    valid_acc = accuracy_score(labels[split_idx['valid']].cpu(), preds[split_idx['valid']].cpu())
    test_acc = accuracy_score(labels[split_idx['test']].cpu(), preds[split_idx['test']].cpu())
    return train_acc, valid_acc, test_acc

In [ ]:
print(sampled_ids)
print(mag_train_idx['paper'])

## 3.2 Then we split the train dataset, validation dataset and test dataset

In [ ]:
import torch

# 获取所有节点的年份信息
years = sampled_graph.ndata['year'].flatten()  # 将年份信息拉平为一维张量

# 使用 PyTorch 或 NumPy 统计年份分布
unique_years, year_counts = torch.unique(years, return_counts=True)

# 打印年份统计结果
print("Year Distribution:")
for year, count in zip(unique_years.tolist(), year_counts.tolist()):
    print(f"Year: {year}, Count: {count}")


# 按年份划分训练集、验证集和测试集
def split_by_year(graph, years, train_cutoff, valid_cutoff):
    # 创建布尔掩码
    train_mask = (years <= train_cutoff)  # 截止到 train_cutoff 年份属于训练集
    valid_mask = (years > train_cutoff) & (years <= valid_cutoff)  # 在 valid_cutoff 年份之间属于验证集
    test_mask = (years > valid_cutoff)  # 剩余的属于测试集

    # 将掩码添加到图中
    graph.ndata['train_mask'] = train_mask
    graph.ndata['valid_mask'] = valid_mask
    graph.ndata['test_mask'] = test_mask

    return graph

# 假设训练集为 2015 年及以前，验证集为 2016-2017 年，测试集为 2018 年及以后
train_cutoff_year = 2013
valid_cutoff_year = 2015

# 更新 sampled_graph 的掩码
sampled_graph = split_by_year(sampled_graph, years, train_cutoff_year, valid_cutoff_year)

# 验证划分结果
print("Number of training nodes:", sampled_graph.ndata['train_mask'].sum().item())
print("Number of validation nodes:", sampled_graph.ndata['valid_mask'].sum().item())
print("Number of testing nodes:", sampled_graph.ndata['test_mask'].sum().item())


## 3.3 Finally we start training

In [ ]:
def main_sampled_graph(sampled_graph):
    # Set up training parameters
    features = sampled_graph.ndata['feat']

    #notice the shape of the labels. This better be changed during the create of the dataset, I forgot. So need to do this tansform every time.
    sampled_labels = sampled_graph.ndata['y']
    sampled_labels = sampled_labels.squeeze(1)  # Convert shape from [20000, 1] to [20000]
    #sampled_graph.ndata['label'] = sampled_graph.ndata['y'].squeeze(1) #we don't change the original dataset
    # Extract masks
    train_idx = torch.nonzero(sampled_graph.ndata['train_mask'], as_tuple=True)[0]
    valid_idx = torch.nonzero(sampled_graph.ndata['valid_mask'], as_tuple=True)[0]
    test_idx = torch.nonzero(sampled_graph.ndata['test_mask'], as_tuple=True)[0]
    split_idx = {'train': train_idx, 'valid': valid_idx, 'test': test_idx}

    in_feats = features.shape[1]
    hidden_feats = 512
    out_feats = sampled_labels.max().item() + 1
    num_layers = 3
    dropout = 0.3

    model = GraphSAGE(in_feats, hidden_feats, out_feats, num_layers, dropout)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

    print("Features shape:", features.shape)
    print("Sampled Labels shape:", sampled_labels.shape)
    print("Train indices shape:", train_idx.shape)

    # Training loop
    for epoch in range(200):  # Example: 100 epochs
        loss = train(model, sampled_graph, features, sampled_labels, train_idx, optimizer)
        train_acc, valid_acc, test_acc = evaluate(model, sampled_graph, features, sampled_labels, split_idx)
        print(f'Epoch {epoch + 1}: Loss = {loss:.4f}, Train Acc = {train_acc:.4f}, '
                  f'Valid Acc = {valid_acc:.4f}, Test Acc = {test_acc:.4f}')

main_sampled_graph(sampled_graph)

Features shape: torch.Size([200000, 128])
Sampled Labels shape: torch.Size([200000])
Train indices shape: torch.Size([128833])


/usr/local/lib/python3.11/dist-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Epoch 1: Loss = 6.0597, Train Acc = 0.1116, Valid Acc = 0.1648, Test Acc = 0.1675
Epoch 2: Loss = 5.7003, Train Acc = 0.0704, Valid Acc = 0.1335, Test Acc = 0.1860
Epoch 3: Loss = 4.9591, Train Acc = 0.1325, Valid Acc = 0.1517, Test Acc = 0.1962
Epoch 4: Loss = 5.0384, Train Acc = 0.1325, Valid Acc = 0.1164, Test Acc = 0.1468
Epoch 5: Loss = 4.9779, Train Acc = 0.1291, Valid Acc = 0.1071, Test Acc = 0.1311
Epoch 6: Loss = 4.8602, Train Acc = 0.1653, Valid Acc = 0.1860, Test Acc = 0.2299
Epoch 7: Loss = 4.7279, Train Acc = 0.1696, Valid Acc = 0.2422, Test Acc = 0.3161
Epoch 8: Loss = 4.5972, Train Acc = 0.1642, Valid Acc = 0.2378, Test Acc = 0.3102
Epoch 9: Loss = 4.4891, Train Acc = 0.1672, Valid Acc = 0.2379, Test Acc = 0.3082
Epoch 10: Loss = 4.4044, Train Acc = 0.1818, Valid Acc = 0.2468, Test Acc = 0.3196
Epoch 11: Loss = 4.3036, Train Acc = 0.2003, Valid Acc = 0.2477, Test Acc = 0.3153
Epoch 12: Loss = 4.2254, Train Acc = 0.2012, Valid Acc = 0.2144, Test Acc = 0.2642
Epoch 13: Los

In [ ]:
print(sampled_graph.ndata['y'].shape)

# Fourth Section: similarly we train the graphsage on arxiv dataset using original embedding

In [ ]:
#This one is for cpu
import torch
import torch.nn.functional as F
import dgl
from dgl.nn import SAGEConv
from ogb.nodeproppred import DglNodePropPredDataset
from sklearn.metrics import accuracy_score

#This below is the same with the function in section 3
# 定义 GraphSAGE 模型
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats, num_layers, dropout):
        super(GraphSAGE, self).__init__()
        self.layers = torch.nn.ModuleList()
        self.layers.append(SAGEConv(in_feats, hidden_feats, aggregator_type='mean'))
        for _ in range(num_layers - 2):
            self.layers.append(SAGEConv(hidden_feats, hidden_feats, aggregator_type='mean'))
        self.layers.append(SAGEConv(hidden_feats, out_feats, aggregator_type='mean'))
        self.dropout = dropout

    def forward(self, g, x):
        for i, layer in enumerate(self.layers):
            x = layer(g, x)
            if i != len(self.layers) - 1:  # 最后一层不使用激活
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

# 训练函数
def train(model, graph, features, labels, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(graph, features)
    loss = F.cross_entropy(out[train_idx], labels[train_idx], label_smoothing=0.1)  # 添加标签平滑
    loss.backward()
    optimizer.step()
    return loss.item()

# 验证/测试函数
@torch.no_grad()
def evaluate(model, graph, features, labels, split_idx):
    model.eval()
    out = model(graph, features)
    preds = out.argmax(dim=1)

    train_acc = accuracy_score(labels[split_idx['train']].cpu(), preds[split_idx['train']].cpu())
    valid_acc = accuracy_score(labels[split_idx['valid']].cpu(), preds[split_idx['valid']].cpu())
    test_acc = accuracy_score(labels[split_idx['test']].cpu(), preds[split_idx['test']].cpu())

    return train_acc, valid_acc, test_acc

#Only the main function changed
# 主函数
def main():
    # 1. 加载 ogbn-arxiv 数据集
    arxiv_dataset = DglNodePropPredDataset(name="ogbn-arxiv")
    arxiv_split_idx = arxiv_dataset.get_idx_split()
    arxiv_train_idx, arxiv_valid_idx, arxiv_test_idx = (
        arxiv_split_idx["train"], arxiv_split_idx["valid"], arxiv_split_idx["test"]
    )
    arxiv_graph, arxiv_label = arxiv_dataset[0]  # 图和标签

    # 2. 数据预处理
    arxiv_graph.ndata['label'] = arxiv_label.squeeze(1)  # 将标签存储到图节点数据中
    arxiv_graph = dgl.to_bidirected(arxiv_graph, copy_ndata=True)  # 转换为无向图
    arxiv_graph = dgl.add_self_loop(arxiv_graph)  # 添加自环
    arxiv_graph.create_formats_()  # 转换为稀疏格式，加速计算

    # 3. 设置特征和标签
    features = arxiv_graph.ndata['feat']
    labels = arxiv_graph.ndata['label']
    split_idx = {
        'train': arxiv_train_idx,
        'valid': arxiv_valid_idx,
        'test': arxiv_test_idx,
    }

    # 4. 将数据移到 GPU（如果可用）
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    features = features.to(device)
    labels = labels.to(device)
    arxiv_graph = arxiv_graph.to(device)

    # 5. 初始化模型和优化器
    in_feats = features.shape[1]
    hidden_feats = 512  # 隐藏层维度
    out_feats = labels.max().item() + 1  # 输出类别数
    num_layers = 3  # GraphSAGE 层数
    dropout = 0.3

    model = GraphSAGE(in_feats, hidden_feats, out_feats, num_layers, dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

    # 6. 训练和评估
    for epoch in range(200):  # 示例：200 轮训练
        loss = train(model, arxiv_graph, features, labels, split_idx['train'], optimizer)
        train_acc, valid_acc, test_acc = evaluate(model, arxiv_graph, features, labels, split_idx)

        print(f"Epoch {epoch + 1}: Loss = {loss:.4f}, "
              f"Train Acc = {train_acc:.4f}, Valid Acc = {valid_acc:.4f}, Test Acc = {test_acc:.4f}")

if __name__ == "__main__":
    main()


In [ ]:
#Output training process

# Fifth Section:now we can train the graphsage using the 2 dataset together

## 5.1 Align feature dimension



In [ ]:
import torch
import dgl

# 对齐特征维度, 如果维度一样可以不用对齐
def align_feature_dimension(graph1, graph2):
    feat_dim1 = graph1.ndata['feat'].shape[1]
    feat_dim2 = graph2.ndata['feat'].shape[1]

    # 如果维度不相等，补齐特征 #都是128维度，所以不用对齐
    if feat_dim1 < feat_dim2:
        pad = torch.zeros(graph1.num_nodes(), feat_dim2 - feat_dim1).to(graph1.ndata['feat'].device)
        graph1.ndata['feat'] = torch.cat([graph1.ndata['feat'], pad], dim=1)
    elif feat_dim2 < feat_dim1:
        pad = torch.zeros(graph2.num_nodes(), feat_dim1 - feat_dim2).to(graph2.ndata['feat'].device)
        graph2.ndata['feat'] = torch.cat([graph2.ndata['feat'], pad], dim=1)

    return graph1, graph2

#sampled_graph, arxiv_graph = align_feature_dimension(sampled_graph, arxiv_graph)
#print(sampled_graph.ndata['feat'].shape)
#print(arxiv_graph.ndata['feat'].shape)

## 5.2 Add label to 2 graph, add ID to arxiv_graph, for sampled_graph, there has been ID.

In [ ]:
#由于前面处理了label数据，现在需要判断前面是否处理过，如果没处理过则进行处理 如果运行了这个再运行前面的训练代码会报错，因为label已经被处理过了
# 检查 arxiv_graph 中是否存在 'label' 数据，如果不存在就赋值
if 'label' not in arxiv_graph.ndata or arxiv_graph.ndata['label'] is None:
    arxiv_graph.ndata['label'] = arxiv_label

# 检查 sampled_graph 中是否存在 'y' 数据，如果不存在就赋值
if 'y' not in sampled_graph.ndata or sampled_graph.ndata['y'] is None:
    sampled_graph.ndata['y'] = sampled_graph.ndata['y']


# 检查 arxiv_label 是否是一维向量
if arxiv_label.dim() > 1:
    arxiv_graph.ndata['label'] = arxiv_label.squeeze(1)

# 检查 sampled_graph 的 y 是否是一维向量
if sampled_graph.ndata['y'].dim() > 1:
    sampled_graph.ndata['y'] = sampled_graph.ndata['y'].squeeze(1)

print(arxiv_graph.ndata['label'])
print(sampled_graph.ndata['y'])

In [ ]:
# 为 arxiv_graph 添加原始 ID
arxiv_graph.ndata['_ID'] = torch.arange(arxiv_graph.num_nodes(), dtype=torch.int64)

# 验证 _ID 是否添加成功
print("Node IDs (_ID):", arxiv_graph.ndata['_ID'][:20])  # 打印前10个节点的 ID

print(sampled_graph.ndata['_ID'][:20])

## 5.3 merge 2 graph, node and edge;

1)store attritute "source" to identity the source of the node； (Graph1 ：sampld_graph节点来源标记为 0; Graph2 ：arxiv_graph节点来源标记为 1. )

2)store sttribute "_ID" to reserve the nodeID for mapping to the paperId.



In [ ]:
print(sampled_graph)
print(arxiv_graph)

In [ ]:
# 合并两个图
def merge_graphs(graph1, graph2):
    import torch
    # 合并节点特征
    #cat中的dim参数表示第0维合并，第1维，128维度保持不变
    print(graph1.ndata['feat'].shape)
    print(graph2.ndata['feat'].shape)
    combined_feat = torch.cat([graph1.ndata['feat'], graph2.ndata['feat']], dim=0)
    print(combined_feat.shape)

    import torch.nn.functional as F

    def combine_and_one_hot_encode(graph1, graph2):
        # 获取标签
        labels1 = graph1.ndata['y']
        labels2 = graph2.ndata['label']

        # 合并标签，将两个图的标签范围扩展为 [0, total_classes)
        total_classes = labels1.max().item() + labels2.max().item() + 2  # 总类别数为两个图的最大类别+1
        print(labels1)
        print(labels2)
        combined_labels = torch.cat([labels1, labels2], dim=0)
        print(combined_labels)

        # 进行 one-hot 编码
        one_hot_encoded_labels = F.one_hot(combined_labels, num_classes=total_classes)
        print(one_hot_encoded_labels[-1])

        return one_hot_encoded_labels

    # 合并并进行编码
    combined_labels = combine_and_one_hot_encode(graph1, graph2)
    print("One-hot Encoded Labels:")
    print(combined_labels[3])

    # 合并年份信息
    combined_year = torch.cat([graph1.ndata['year'], graph2.ndata['year']], dim=0)

    # 合并原始节点 ID
    combined_id = torch.cat([graph1.ndata['_ID'], graph2.ndata['_ID']], dim=0)

    #添加来源标签
    graph1_source = torch.zeros(graph1.num_nodes(), dtype=torch.int64)  # Graph1 ：sampld_graph节点来源标记为 0
    graph2_source = torch.ones(graph2.num_nodes(), dtype=torch.int64)   # Graph2 ：arxiv_graph节点来源标记为 1
    combined_source = torch.cat([graph1_source, graph2_source], dim=0)


    # 合并标签
    # 合并边信息
    graph1_edges = graph1.edges()
    graph2_edges = graph2.edges()

    # 合并边 这里对graph1的起始点做了位置偏移，所以合并后的边没有重合的
    combined_src = torch.cat([graph1_edges[0], graph2_edges[0] + graph1.num_nodes()])
    combined_dst = torch.cat([graph1_edges[1], graph2_edges[1] + graph1.num_nodes()])


    # 创建新图
    combined_graph = dgl.graph((combined_src, combined_dst))
    combined_graph.ndata['feat'] = combined_feat
    combined_graph.ndata['label'] = combined_labels
    combined_graph.ndata['graph_source'] = combined_source  # 添加来源标签
    combined_graph.ndata['year'] = combined_year
    combined_graph.ndata['_ID'] = combined_id

    print("Graph 1 edges:", graph1.num_edges())
    print("Graph 2 edges:", graph2.num_edges())
    print("Combined graph edges:", combined_graph.num_edges())
    print("Combined graph nodes:", combined_graph.num_nodes())
    print("Sample of graph_source:", combined_graph.ndata['graph_source'][:10])
    print("Sample of years:", combined_graph.ndata['year'][:10])
    print("Sample of _ID:", combined_graph.ndata['_ID'][:10])

    return combined_graph

combined_graph = merge_graphs(sampled_graph, arxiv_graph)

In [ ]:
print(combined_graph.ndata["_ID"].shape)
print(combined_graph.ndata["graph_source"].shape)

In [ ]:
def access_original_node(combined_graph, node_id, sampled_graph, arxiv_graph):
    # 确定节点来源
    print("hi")
    graph_source = combined_graph.ndata['graph_source'][node_id].item()  # 来源标识
    print(graph_source)
    print(combined_graph.ndata)
    original_id = combined_graph.ndata['_ID'][node_id] # 原始 ID
    print(original_id)

    if graph_source == 0:
        print(f"Node {node_id} comes from 'sampled_graph', original ID: {original_id}")

    elif graph_source == 1:
        print(f"Node {node_id} comes from 'arxiv_graph', original ID: {original_id}")


# 访问第 0 个节点的数据
access_original_node(combined_graph, node_id=2, sampled_graph=sampled_graph, arxiv_graph=arxiv_graph)

# 访问第 200,000 个节点的数据
access_original_node(combined_graph, node_id=200003, sampled_graph=sampled_graph, arxiv_graph=arxiv_graph)


## 5.4 Add the masks attribute to the combined graph

remeber the order id of the node in the combined graph can't be seen as the masking id, when we are doing the mask id, always think about the _ID attribute.

One way to check this is to output the year of the train node, in arxiv dataset, they are less than 2018, while in the sampled dataset, they are less than 2013.



In [ ]:
graph1 = sampled_graph
graph2 = arxiv_graph
#存储原始状态

In [ ]:
def check_and_initialize_masks(sampled_graph, arxiv_graph):
    # 处理 sampled_graph
    if 'train_mask' not in sampled_graph.ndata or 'valid_mask' not in sampled_graph.ndata or 'test_mask' not in sampled_graph.ndata:
        print("Initializing masks for sampled_graph...")
        years = sampled_graph.ndata['year'].flatten()  # 获取年份信息
        print(years)
        train_cutoff_year = 2013
        valid_cutoff_year = 2015

        # 划分训练集、验证集和测试集
        sampled_graph = split_by_year(sampled_graph, years, train_cutoff_year, valid_cutoff_year)

        # 验证划分结果
        print("Number of training nodes:", sampled_graph.ndata['train_mask'].sum().item())
        print("Number of validation nodes:", sampled_graph.ndata['valid_mask'].sum().item())
        print("Number of testing nodes:", sampled_graph.ndata['test_mask'].sum().item())
    else:
        print("Masks for sampled_graph already initialized!")

    # 处理 arxiv_graph
    if 'train_mask' not in arxiv_graph.ndata or 'valid_mask' not in arxiv_graph.ndata or 'test_mask' not in arxiv_graph.ndata:
        print("Initializing masks for arxiv_graph...")
        # 加载 ogbn-arxiv 数据集
        arxiv_dataset = DglNodePropPredDataset(name="ogbn-arxiv")
        arxiv_split_idx = arxiv_dataset.get_idx_split()
        arxiv_train_idx, arxiv_valid_idx, arxiv_test_idx = (
            arxiv_split_idx["train"], arxiv_split_idx["valid"], arxiv_split_idx["test"]
        )
        #arxiv_graph.ndata['label'] = arxiv_graph.ndata['label'].squeeze(1)  # 确保标签是一维向量

        # 设置掩码
        arxiv_graph.ndata['train_mask'] = torch.zeros(arxiv_graph.num_nodes(), dtype=torch.bool)
        arxiv_graph.ndata['valid_mask'] = torch.zeros(arxiv_graph.num_nodes(), dtype=torch.bool)
        arxiv_graph.ndata['test_mask'] = torch.zeros(arxiv_graph.num_nodes(), dtype=torch.bool)

        print(arxiv_train_idx)

        # 根据划分索引设置掩码
        arxiv_graph.ndata['train_mask'][arxiv_train_idx] = True #在对应的点设为mask， 这里的idx是直接的索引
        arxiv_graph.ndata['valid_mask'][arxiv_valid_idx] = True
        arxiv_graph.ndata['test_mask'][arxiv_test_idx] = True

        print(arxiv_graph.ndata['train_mask'])

        # 验证划分结果
        print("Number of training nodes:", arxiv_graph.ndata['train_mask'].sum().item())
        print("Number of validation nodes:", arxiv_graph.ndata['valid_mask'].sum().item())
        print("Number of testing nodes:", arxiv_graph.ndata['test_mask'].sum().item())
    else:
        print("Masks for arxiv_graph already initialized!")

    return sampled_graph, arxiv_graph

# 使用分割函数
def split_by_year(graph, years, train_cutoff, valid_cutoff):
    # 创建布尔掩码
    train_mask = (years <= train_cutoff)  # 截止到 train_cutoff 年份属于训练集
    valid_mask = (years > train_cutoff) & (years <= valid_cutoff)  # 在 valid_cutoff 年份之间属于验证集
    test_mask = (years > valid_cutoff)  # 剩余的属于测试集

    print(train_mask)
    print(valid_mask)
    print(test_mask)

    # 将掩码添加到图中
    graph.ndata['train_mask'] = train_mask
    graph.ndata['valid_mask'] = valid_mask
    graph.ndata['test_mask'] = test_mask

    return graph




In [ ]:
sampled_graph, arxiv_graph = check_and_initialize_masks(sampled_graph,arxiv_graph)

In [ ]:
#sampled_graph = graph1
#arxiv_graph = graph2

In [ ]:
print(sampled_graph)
print(arxiv_graph)

In [ ]:
# 合并掩码
def merge_masks(graph1, graph2):

    train_mask = torch.cat([graph1.ndata['train_mask'], graph2.ndata['train_mask']], dim=0)
    valid_mask = torch.cat([graph1.ndata['valid_mask'], graph2.ndata['valid_mask']], dim=0)
    test_mask = torch.cat([graph1.ndata['test_mask'], graph2.ndata['test_mask']], dim=0)

    return train_mask, valid_mask, test_mask

# 合并掩码
combined_train_mask, combined_valid_mask, combined_test_mask = merge_masks(sampled_graph, arxiv_graph)

In [ ]:
print(combined_train_mask[-2000:])

print(arxiv_graph.ndata['train_mask'][:100]) # for arxiv , the paper is sorted by year already, so the front are training , the last are testing

print(combined_train_mask[:10])
print(sampled_graph.ndata['train_mask'][:10])

In [ ]:
# 对齐特征维度 如果维度一样可以不用对齐
#sampled_graph, arxiv_graph = align_feature_dimension(sampled_graph, arxiv_graph)

# 合并两个图
#combined_graph = merge_graphs(sampled_graph, arxiv_graph)

# 合并掩码
#combined_train_mask, combined_valid_mask, combined_test_mask = merge_masks(sampled_graph, arxiv_graph)


# 添加合并后的掩码到图中
combined_graph.ndata['train_mask'] = combined_train_mask
combined_graph.ndata['valid_mask'] = combined_valid_mask
combined_graph.ndata['test_mask'] = combined_test_mask

# 验证结果
print("Number of nodes in combined graph:", combined_graph.num_nodes())
print("Number of edges in combined graph:", combined_graph.num_edges())
print("Number of training nodes:", combined_train_mask.sum().item())
print("Number of validation nodes:", combined_valid_mask.sum().item())
print("Number of testing nodes:", combined_test_mask.sum().item())


## 5.5 save and load the graph

In [ ]:
print(combined_graph)
import pickle

def save_graph(graph, filename):
    with open(filename, 'wb') as f:
        pickle.dump(graph, f)
    print(f"Graph saved to {filename}")

# 保存图并压缩
save_graph(combined_graph, "combined_graph.pkl")

In [ ]:
import gzip
import pickle

def save_graph_compressed(graph, filename):
    # 使用 gzip 压缩保存图
    with gzip.open(filename, 'wb') as f:
        pickle.dump(graph, f)
    print(f"Graph saved and compressed to {filename}")

def load_graph_compressed(filename):
    # 使用 gzip 解压缩并加载图
    with gzip.open(filename, 'rb') as f:
        graph = pickle.load(f)
    print(f"Graph loaded from {filename}")
    return graph
# 压缩保存图
save_graph_compressed(combined_graph, "combined_graph.pkl.gz")

In [ ]:
# 加载压缩图
combined_graph = load_graph_compressed("combined_graph.pkl.gz")

# 验证加载后的图结构
print("Node data:", combined_graph.ndata.keys())
print("Edge data:", combined_graph.edata.keys())

In [ ]:
print(combined_graph)

## 5.6 Finally train it!

In [ ]:
print(arxiv_label)

In [ ]:
#This one is for cpu
import torch
import torch.nn.functional as F
import dgl
from dgl.nn import SAGEConv
from ogb.nodeproppred import DglNodePropPredDataset
from sklearn.metrics import accuracy_score

#This below is the same with the function in section 3
# 定义 GraphSAGE 模型
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats, num_layers, dropout):
        super(GraphSAGE, self).__init__()
        self.layers = torch.nn.ModuleList()
        self.layers.append(SAGEConv(in_feats, hidden_feats, aggregator_type='mean'))
        for _ in range(num_layers - 2):
            self.layers.append(SAGEConv(hidden_feats, hidden_feats, aggregator_type='mean'))
        self.layers.append(SAGEConv(hidden_feats, out_feats, aggregator_type='mean'))
        self.dropout = dropout

    def forward(self, g, x):
        for i, layer in enumerate(self.layers):
            x = layer(g, x)
            if i != len(self.layers) - 1:  # 最后一层不使用激活
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

# 训练函数
def train(model, graph, features, labels, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    labels = labels.float()
    out = model(graph, features)
    loss = F.cross_entropy(out[train_idx], labels[train_idx], label_smoothing=0.1)  # 添加标签平滑
    loss.backward()
    optimizer.step()
    return loss.item()

# 验证/测试函数
@torch.no_grad()
def evaluate(model, graph, features, labels, split_idx):
    model.eval()
    out = model(graph, features)
    preds = out.argmax(dim=1)

    # 转换标签为类别索引形式
    labels_idx = labels.argmax(dim=1)  # 从 one-hot 编码转为索引形式

    train_acc = accuracy_score(labels_idx[split_idx['train']].cpu(), preds[split_idx['train']].cpu())
    valid_acc = accuracy_score(labels_idx[split_idx['valid']].cpu(), preds[split_idx['valid']].cpu())
    test_acc = accuracy_score(labels_idx[split_idx['test']].cpu(), preds[split_idx['test']].cpu())

    return train_acc, valid_acc, test_acc

In [ ]:
import torch
import dgl
import os

def batchify(graph, train_idx, batch_size):
    # 将训练节点划分为批次
    for i in range(0, len(train_idx), batch_size):
        batch_nodes = train_idx[i:i + batch_size]
        subgraph = dgl.node_subgraph(graph, batch_nodes)  # 创建子图
        yield subgraph, batch_nodes


# 提取特征和标签
features = combined_graph.ndata['feat']
labels = combined_graph.ndata['label']
train_idx = torch.nonzero(combined_graph.ndata['train_mask'], as_tuple=True)[0]
valid_idx = torch.nonzero(combined_graph.ndata['valid_mask'], as_tuple=True)[0]
test_idx = torch.nonzero(combined_graph.ndata['test_mask'], as_tuple=True)[0]

split_idx = {'train': train_idx, 'valid': valid_idx, 'test': test_idx}

# 将数据移动到合适的设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
features = features.to(device)
labels = labels.to(device)
combined_graph = combined_graph.to(device)

# 初始化模型和优化器
in_feats = features.shape[1]
hidden_feats = 512
out_feats = labels.shape[1]  # 输出类别数 (来自 one-hot 编码)
num_layers = 3
dropout = 0.3

model = GraphSAGE(in_feats, hidden_feats, out_feats, num_layers, dropout).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# 创建保存路径
save_dir = "./saved_models"
os.makedirs(save_dir, exist_ok=True)

# 定义批次大小
batch_size = 1024

best_valid_acc = 0
for epoch in range(200):
    model.train()
    total_loss = 0

    # 手动处理每个批次
    for subgraph, batch_nodes in batchify(combined_graph, train_idx, batch_size):
        subgraph = subgraph.to(device)  # 将子图移动到设备
        batch_feats = subgraph.ndata['feat']
        batch_labels = subgraph.ndata['label']

        optimizer.zero_grad()
        logits = model(subgraph, batch_feats)
        loss = -(batch_labels * torch.log_softmax(logits, dim=1)).sum(dim=1).mean()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # 验证和测试
    train_acc, valid_acc, test_acc = evaluate(model, combined_graph, features, labels,
                                              {'train': train_idx, 'valid': valid_idx, 'test': test_idx})

    print(f"Epoch {epoch + 1}: Loss = {total_loss:.4f}, "
          f"Train Acc = {train_acc:.4f}, Valid Acc = {valid_acc:.4f}, Test Acc = {test_acc:.4f}")

    # 保存验证集上最好的模型
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model_path = os.path.join(save_dir, "best_model.pth")
        torch.save(model.state_dict(), best_model_path)
        print(f"Best model saved at epoch {epoch + 1}: {best_model_path}")

    # 每 50 个 epoch 保存一次模型
    if (epoch + 1) % 50 == 0:
        model_save_path = os.path.join(save_dir, f"model_epoch_{epoch + 1}.pth")
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved at epoch {epoch + 1}: {model_save_path}")

In [ ]:
# 清理垃圾收集器
import gc
gc.collect()

# Other session, may used when using graphalign embedding

In [ ]:
#现在要用e5编码我的paper了，如果我还是用标题和摘要呢？

# 加载你生成的 E5 编码（假设形状为 [num_papers, 384]）
paper_embeddings = np.load('paper_embeddings.npy')
paper_embeddings = torch.tensor(paper_embeddings, dtype=torch.float32)
graph.nodes['paper'].data['feat'] = paper_embeddings

# 为 author 节点生成随机初始特征（可替换为其他方法）??不要随机初始特征吧
graph.nodes['author'].data['feat'] = torch.randn(graph.num_nodes('author'), 128)
graph.nodes['institution'].data['feat'] = torch.randn(graph.num_nodes('institution'), 128)
graph.nodes['field_of_study'].data['feat'] = torch.randn(graph.num_nodes('field_of_study'), 128)

"""加入不使用随机初始化
# 方法1：作者特征 = 其撰写论文的 E5 特征均值
# ---------------------------------------------------
# 获取作者与论文的边关系（'author' -- 'writes' --> 'paper'）
author_paper_edges = graph.edges(etype=('author', 'writes', 'paper'))
author_ids, paper_ids = author_paper_edges[0], author_paper_edges[1]

# 计算每个作者的特征（聚合其所有论文的 E5 嵌入）
author_feat = torch.zeros(graph.num_nodes('author'), paper_embeddings.shape[1])
for author_id in torch.unique(author_ids):
    mask = author_ids == author_id
    author_feat[author_id] = paper_embeddings[paper_ids[mask]].mean(dim=0)

graph.nodes['author'].data['feat'] = author_feat

# 方法2：机构特征 = 其下属作者的特征均值
# ---------------------------------------------------
# 获取机构与作者的边关系（'author' -- 'affiliated_with' --> 'institution'）
author_inst_edges = graph.edges(etype=('author', 'affiliated_with', 'institution'))
author_ids, inst_ids = author_inst_edges[0], author_inst_edges[1]

# 计算每个机构的特征（聚合其所有作者的特征）
institution_feat = torch.zeros(graph.num_nodes('institution'), author_feat.shape[1])
for inst_id in torch.unique(inst_ids):
    mask = inst_ids == inst_id
    institution_feat[inst_id] = author_feat[author_ids[mask]].mean(dim=0)

graph.nodes['institution'].data['feat'] = institution_feat

# 方法3：研究领域特征 = 其相关论文的 E5 特征均值
# ---------------------------------------------------
# 获取论文与研究领域的边关系（'paper' -- 'has_topic' --> 'field_of_study'）
paper_field_edges = graph.edges(etype=('paper', 'has_topic', 'field_of_study'))
paper_ids, field_ids = paper_field_edges[0], paper_field_edges[1]

# 计算每个领域的特征（聚合其相关论文的 E5 嵌入）
field_feat = torch.zeros(graph.num_nodes('field_of_study'), paper_embeddings.shape[1])
for field_id in torch.unique(field_ids):
    mask = field_ids == field_id
    field_feat[field_id] = paper_embeddings[paper_ids[mask]].mean(dim=0)

graph.nodes['field_of_study'].data['feat'] = field_feat

"""


In [ ]:
import pandas as pd
paper_mapping = pd.read_csv("/content/dataset/ogbn_mag/mapping/paper_entidx2name.csv.gz", compression = "gzip")
print(paper_mapping) #这是nodeid到mag papaer id的映射

In [ ]:
import pandas as pd
author_mapping = pd.read_csv("/content/dataset/ogbn_mag/mapping/author_entidx2name.csv.gz", compression = "gzip")
print(author_mapping) #这是nodeid到author id的映射

In [ ]:
print(graph,label)

In [ ]:
#GraphSAGE是为同构图设计的，无法直接处理异构图的多类型节点和边
#分类型单独应用GraphSAGE，再聚合结果
#调整后的GraphSAGE模型
from dgl.nn import SAGEConv
import torch.nn as nn

class HeteroGraphSAGE(nn.Module):
    def __init__(self, in_feats_dict, hid_feats, out_feats):
        super().__init__()
        # 定义每个节点类型的投影层
        self.proj = nn.ModuleDict({
            ntype: nn.Linear(in_feats_dict[ntype], hid_feats)
            for ntype in in_feats_dict
        })
        # 定义异构卷积层（分边类型处理）
        self.conv1 = dgl.nn.HeteroGraphConv({
            rel: SAGEConv(hid_feats, hid_feats, 'mean')
            for rel in graph.etypes
        })
        self.classifier = nn.Linear(hid_feats, out_feats)

    def forward(self, graph):
        # 投影所有节点特征到 hid_feats
        h_dict = {ntype: self.proj[ntype](graph.nodes[ntype].data['feat'])
                  for ntype in graph.ntypes}
        # 异构卷积
        h_dict = self.conv1(graph, h_dict)
        h_dict = {k: nn.functional.relu(v) for k, v in h_dict.items()}
        # 分类（仅 paper 节点）
        return self.classifier(h_dict['paper'])

In [ ]:
#训练与苹

In [ ]:
#用不到，这是用来保存图的
import dgl
import torch
import pickle
from ogb.nodeproppred import DglNodePropPredDataset

# 加载数据
dataset = DglNodePropPredDataset(name="ogbn-arxiv")
split_idx = dataset.get_idx_split()
graph, labels = dataset[0]

# 方法1：保存为pickle
data_to_save = {"graph": graph, "labels": labels, "split_idx": split_idx}
with open("arxiv_data.pkl", "wb") as f:
    pickle.dump(data_to_save, f)

# 方法2：使用DGL内置函数
dgl.save_graphs("arxiv_graph.dgl", [graph])
torch.save({"labels": labels, "split_idx": split_idx}, "arxiv_meta.pt")

print("数据保存成功！")

In [ ]:
!ls -lh

In [ ]:
#evaluator
from ogb.graphproppred import Evaluator

evaluator = Evaluator(name = "ogbg-mag")
# You can learn the input and output format specification of the evaluator as follows.
# print(evaluator.expected_input_format)
# print(evaluator.expected_output_format)
input_dict = {"y_true": y_true, "y_pred": y_pred}
result_dict = evaluator.eval(input_dict) # E.g., {"rocauc": 0.7321}

In [ ]:
#测试embedding，但是跑不起来
import argparse
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from ogb.nodeproppred import DglNodePropPredDataset
import wget
import shutil
import gzip


MODEL_NAME = {"e5": "intfloat/e5-small-v2", "ofa": "../../cache/transformer-model/multi-qa-distilbert-cos-v1"}
short_name = {"ogbn-papers100M": "100M", "ogbn-arxiv": "arxiv", "ogbn-products": "products"}


def decompress_gz(file_path, output_path):
    with gzip.open(file_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"Decompressed file to {output_path}")


class Ogb_dataset(Dataset):
    def __init__(self, datas):
        self.data = datas
        self.length = len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.length


class Tokenizer(object):
    def __init__(self, tokenizer, args):
        super(Tokenizer, self).__init__()
        self.max_token_len = args.max_token_len
        self.tokenizer = tokenizer
        self.padding = "max_length"
        self.truncation = True

    def __call__(self, examples):
        if isinstance(examples, str):
            return self.tokenizer(examples, padding=self.padding, truncation=self.truncation,
                                  max_length=self.max_token_len, return_tensors="pt")
        else:
            return self.tokenizer(examples["text"], padding=self.padding, truncation=self.truncation,
                                  max_length=self.max_token_len, return_tensors="pt")


class Gen_ogb_data():
    def __init__(self, args):
        self.args = args
        self.download_raw_data(args.dataset_name)
        self.get_text_data(args.dataset_name)
        self.get_emb(args.dataset_name)

    def average_pool(self, last_hidden_states, attention_mask):
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

    def download_raw_data(self, dataset_name="ogbn-arxiv"):
        os.makedirs(self.args.data_save_path, exist_ok=True)
        dataset_path = os.path.join(self.args.data_save_path, dataset_name)
        os.makedirs(dataset_path, exist_ok=True)
        if dataset_name == "ogbn-arxiv":
            if not os.path.exists(os.path.join(dataset_path, "titleabs.tsv")):
                url = "https://snap.stanford.edu/ogb/data/misc/ogbn_arxiv/titleabs.tsv.gz"
                wget.download(url, os.path.join(dataset_path, "titleabs.tsv.gz"))
                decompress_gz(os.path.join(dataset_path, "titleabs.tsv.gz"), os.path.join(dataset_path, "titleabs.tsv"))
                os.remove(os.path.join(dataset_path, "titleabs.tsv.gz"))
            self.dgl_dataset = DglNodePropPredDataset(dataset_name, root=os.path.join(self.args.data_save_path,
                                                                                      "ogb-official-data"))

    def get_text_data(self, dataset_name="ogbn-arxiv"):
        dataset_path = os.path.join(self.args.data_save_path, dataset_name)
        ogbn_official_path = os.path.join(self.args.data_save_path, "ogb-official-data")

        if dataset_name == "ogbn-arxiv":
            self.df = pd.read_csv(os.path.join(dataset_path, 'titleabs.tsv'), sep='\t')
            decompress_gz(os.path.join(ogbn_official_path, "ogbn_arxiv", "mapping", 'nodeidx2paperid.csv.gz'),
                          os.path.join(ogbn_official_path, "ogbn_arxiv", "mapping", 'nodeidx2paperid.csv'))
            self.nodeid2contentid = pd.read_csv(
                os.path.join(ogbn_official_path, "ogbn_arxiv", "mapping", 'nodeidx2paperid.csv'))
            self.df.columns = ["paperid", "title", "abs"]
            self.nodeid2contentid.columns = ["nodeid", "paperid"]
            data = pd.merge(self.nodeid2contentid, self.df, how="left", on="paperid")
            Datasets = data.values[:, 2:]

        dataframe = pd.DataFrame(Datasets)
        dataframe.to_csv(os.path.join(dataset_path, f'{dataset_name}_title_content.csv'), index=False)
        print(f"{dataset_name} title_content.csv has been saved!")

    def get_emb(self, dataset_name="ogbn-arxiv"):
        dataset_path = os.path.join(self.args.data_save_path, dataset_name)
        if dataset_name == "ogbn-arxiv":
            Datas = []
            data = pd.read_csv(os.path.join(dataset_path, f'{dataset_name}_title_content.csv')).values
            for k in range(data.shape[0]):
                data_dict = {}
                if pd.isnull(data[k][0]) and pd.isnull(data[k][1]):
                    data_dict["text"] = " .  "
                elif pd.isnull(data[k][1]):
                    data_dict["text"] = data[k][0]
                elif pd.isnull(data[k][0]):
                    data_dict["text"] = data[k][1]
                else:
                    data_dict["text"] = data[k][0] + ". " + data[k][1]
                Datas.append(data_dict)
            text_dataset = Ogb_dataset(Datas)
            text_dataloader = DataLoader(text_dataset, shuffle=False, batch_size=self.args.batch_size)
        else:
            raise ValueError

        if self.args.Model == "e5":
            tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME[self.args.Model])
            model_tokenizer = Tokenizer(tokenizer, self.args)
            model = AutoModel.from_pretrained(MODEL_NAME[self.args.Model])
            model.to(self.args.device)
            model.eval()
            nodes_embed = []
            epoch_iter = tqdm(text_dataloader)
            print(f"Generating {self.args.Model} embedding!")
            with torch.no_grad():
                for batch in epoch_iter:
                    batch = model_tokenizer(batch)
                    batch = {k: v.to(self.args.device) for k, v in batch.items()}
                    outputs = model(**batch)
                    embeddings = self.average_pool(outputs.last_hidden_state, batch['attention_mask'])
                    for i in range(embeddings.shape[0]):
                        nodes_embed.append(embeddings[i].cpu().numpy().astype(self.args.dtype)) #我们这没有这么复杂

            nodes_embed = np.stack(nodes_embed, axis=0)
            np.save(os.path.join(dataset_path, f"{short_name[dataset_name]}_embedding_{self.args.Model}_{self.args.dtype}.npy"),
                    nodes_embed)
            print(f"{short_name[dataset_name]}_embedding_{self.args.Model}_{self.args.dtype}.npy has been saved!")

        elif self.args.Model == "ofa":
            model = SentenceTransformer(MODEL_NAME[self.args.Model])
            model.to(self.args.device)
            model.eval()
            with torch.no_grad():
                texts = []
                for d in Datas:
                    texts.append(d["text"])
                embeddings = model.encode(texts, batch_size=self.args.batch_size, show_progress_bar=True,
                                          convert_to_tensor=False, convert_to_numpy=True)
                np.save(os.path.join(dataset_path, f"{short_name[dataset_name]}_embedding_{self.args.Model}_{self.args.dtype}.npy"),
                        embeddings.astype(self.args.dtype))
            print(f"{short_name[dataset_name]}_embedding_{self.args.Model}_{self.args.dtype}.npy has been saved!")
        else:
            raise ValueError


if __name__ == "__main__":
    # 加载数据集
    dataset = DglNodePropPredDataset(name="ogbn-arxiv")
    split_idx = dataset.get_idx_split()
    train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
    graph, label = dataset[0]

    # 设置参数
    parser = argparse.ArgumentParser(description="")
    parser.add_argument('--data_save_path', type=str, default='./data')
    parser.add_argument('--device', type=str, default='cuda:0')
    parser.add_argument('--Model', type=str, default='e5')
    parser.add_argument('--batch_size', type=int, default=512)
    parser.add_argument('--dataset_name', type=str, default="ogbn-arxiv")
    parser.add_argument('--max_token_len', type=int, default=512)
    parser.add_argument('--dtype', type=str, default="float16")
    args = parser.parse_args()

    # 生成嵌入
    gen_ogb = Gen_ogb_data(args)


In [ ]:
!pip install wget

In [ ]:
#一种很新的训练方法
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import networkx as nx
from dgl.data import AmazonCoBuyComputerDataset
from dgl.nn import SAGEConv
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import os

# Set the DGL backend to PyTorch
os.environ['DGLBACKEND'] = 'pytorch'


# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Load the dataset
dataset = AmazonCoBuyComputerDataset()
g = dataset[0]
labels = g.ndata['label']

# Split the dataset into training/validation/testing sets
n_nodes = g.num_nodes()
indices = np.random.permutation(n_nodes)
train_idx = indices[:int(0.8*n_nodes)]  # 80% for training
val_idx = indices[int(0.8*n_nodes):int(0.9*n_nodes)]  # 10% for validation
test_idx = indices[int(0.9*n_nodes):]  # 10% for testing

# Create training/validation/testing masks
g.ndata['train_mask'] = torch.zeros(n_nodes, dtype=torch.bool)
g.ndata['val_mask'] = torch.zeros(n_nodes, dtype=torch.bool)
g.ndata['test_mask'] = torch.zeros(n_nodes, dtype=torch.bool)
g.ndata['train_mask'][train_idx] = True
g.ndata['val_mask'][val_idx] = True
g.ndata['test_mask'][test_idx] = True

# Define the GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = SAGEConv(in_feats, hid_feats, 'mean')  # First GraphSAGE layer
        self.conv2 = SAGEConv(hid_feats, hid_feats, 'mean')  # Second GraphSAGE layer
        self.classifier = nn.Linear(hid_feats, out_feats)  # Classification layer

    def forward(self, graph, x):
        x = F.relu(self.conv1(graph, x))  # Apply ReLU activation after the first layer
        x = F.relu(self.conv2(graph, x))  # Apply ReLU activation after the second layer
        return self.classifier(x)  # Output layer

# Prepare three types of node features
def prepare_features(g):
    # Feature type (i): All-one vectors
    ones_feat = torch.ones(g.ndata['feat'].shape)

    # Feature type (ii): Original node features
    original_feat = g.ndata['feat']

    # Feature type (iii): Structural features + one-hot encoding
    # Convert to a NetworkX graph
    # nx_g = g.to_networkx().to_undirected() This line would throw an error for multigraphs
    nx_g = nx.Graph(dgl.to_networkx(g.cpu()).to_undirected())
    # Note: nx.Graph function will merge multiple edges between nodes into a single edge,
    # as NetworkX does not support multigraph structures.
    print(nx_g.number_of_edges())  # Should output the original number of undirected edges: 245,778

    # Calculate structural features
    print("Calculating clustering coefficients...")
    clustering = nx.clustering(nx_g)
    print("Calculating degree centrality...")
    degree_cent = nx.degree_centrality(nx_g)
    print("Calculating betweenness centrality...")
    betweenness = nx.betweenness_centrality(nx_g, k=100)  # Sample some nodes to speed up calculation
    print("Calculating eigenvector centrality...")
    eigenvector = nx.eigenvector_centrality(nx_g, max_iter=1000)

    # Collect features and normalize them
    structural = np.array([[clustering[i], degree_cent[i],
                          betweenness[i], eigenvector[i]] for i in range(n_nodes)])
    structural = StandardScaler().fit_transform(structural)

    # Generate one-hot encodings
    one_hot = torch.eye(n_nodes)

    # Concatenate features
    structural_feat = torch.cat([torch.FloatTensor(structural), one_hot], dim=1)

    return {
        'ones': ones_feat,
        'original': original_feat,
        'structural+onehot': structural_feat
    }

features = prepare_features(g)

# Define the training function
def train_model(feature, label, train_mask, val_mask, test_mask, epochs=400):
    model = GraphSAGE(feature.shape[1], 64, dataset.num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer with learning rate 0.01
    criterion = nn.CrossEntropyLoss()  # Cross-entropy loss function

    best_val_acc = 0
    best_model = None
    history = {'loss': [], 'val_acc': []}

    for epoch in range(epochs):
        model.train()
        logits = model(g, feature)
        loss = criterion(logits[train_mask], label[train_mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            pred = logits.argmax(1)
            val_acc = accuracy_score(label[val_mask].numpy(), pred[val_mask].numpy())

        history['loss'].append(loss.item())
        history['val_acc'].append(val_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = model.state_dict().copy()

        if epoch % 50 == 0:
            print(f'Epoch {epoch:03d} | Loss: {loss.item():.4f} | Val Acc: {val_acc:.4f}')

    # Test the best model
    model.load_state_dict(best_model)
    model.eval()
    with torch.no_grad():
        logits = model(g, feature)
        test_acc = accuracy_score(label[test_mask].numpy(),
                                 logits[test_mask].argmax(1).numpy())

    return history, test_acc

# Train and compare three types of features
results = {}
for feat_name in ['ones', 'original', 'structural+onehot']:
    print(f"\n=== Training with {feat_name} features ===")
    feat = features[feat_name]
    history, test_acc = train_model(feat, labels,
                                  g.ndata['train_mask'],
                                  g.ndata['val_mask'],
                                  g.ndata['test_mask'])
    results[feat_name] = {
        'train_loss': history['loss'],
        'val_acc': history['val_acc'],
        'test_acc': test_acc
    }

# Visualize the training process
plt.figure(figsize=(12, 4))
for i, feat in enumerate(results.keys()):
    plt.subplot(1, 2, 1)
    plt.plot(results[feat]['train_loss'], label=f'{feat}')
    plt.subplot(1, 2, 2)
    plt.plot(results[feat]['val_acc'], label=f'{feat}')

plt.subplot(1, 2, 1)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.legend()
plt.subplot(1, 2, 2)
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.tight_layout()
plt.show()

# Output the test results
print("\n=== Final Test Results ===")
for feat in results:
    print(f"{feat.ljust(20)} Test Accuracy: {results[feat]['test_acc']:.4f}")

In [ ]:
#不知道在干什么
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt

# 加载数据集
dataset = PygNodePropPredDataset(name="ogbn-arxiv", root="dataset/")
data = dataset[0]
split_idx = dataset.get_idx_split()

# 数据集划分
train_idx = split_idx["train"]
valid_idx = split_idx["valid"]
test_idx = split_idx["test"]

# 定义GraphSAGE模型
class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# 初始化模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GraphSAGE(data.num_features, 256, dataset.num_classes).to(device)
data = data.to(device)

# 优化器和损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# 训练函数
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[train_idx], data.y[train_idx].squeeze())
    loss.backward()
    optimizer.step()
    return loss.item()

# 验证函数
def evaluate(mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = pred[mask] == data.y[mask].squeeze()
        acc = int(correct.sum()) / int(mask.sum())
    return acc

# 训练过程
train_losses = []
val_accuracies = []
test_accuracy = 0
for epoch in range(100):  # 训练100轮
    loss = train()
    train_losses.append(loss)
    val_acc = evaluate(valid_idx)
    val_accuracies.append(val_acc)

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss:.4f}, Val Accuracy: {val_acc:.4f}")

test_accuracy = evaluate(test_idx)

# 输出测试结果
print(f"Final Test Accuracy: {test_accuracy:.4f}")

# 可视化训练损失和验证精度
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label="Validation Accuracy", color="orange")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()
